In [ ]:
%pip install matplotlib numpy opencv-python scikit-image

In [ ]:
from collections.abc import Generator, Iterator
from cv2 import imread, imwrite
from numpy import concatenate, ndarray
from pathlib import Path
from random import shuffle
from skimage.exposure import match_histograms

import matplotlib.pyplot as plt
import sys


In [ ]:
def max_div(l: int, i: int = 2) -> int | None:
    """max_div(l, i) finds the smallest divisor d (at least i) of l.

    Args:
        l (int): The number to find a divisor of.
        i (int, optional): The minimum value of the divisor. Defaults to 2.

    Returns:
        int | None: The smallest divisor of l, or None if l has no other divisor than itself higher than i.
    """
    while i < l:
        if l % i == 0:
            return i
        i += 1
    return None

In [ ]:
def parts(img: ndarray, dx: int, dy: int) -> Generator[ndarray, None, None]:
    """parts(img, dx, dy) splits img into a grid of images of size dx*dy.

    Args:
        img (ndarray): The image to split.
        dx (int): The width of a splitted image part.
        dy (int): The height of a splitted image part.

    Yields:
        Generator[ndarray, None, None]: The splitted image parts.
    """
    for y in range(0, img.shape[0], dy):
        for x in range(0, img.shape[1], dx):
            yield img[y:y+dy, x:x+dx]

In [ ]:
def rebuild(tiles: list[ndarray], nx: int) -> ndarray:
    """rebuild(tiles, nx) recreates an image from a grid of images of the same size.

    Args:
        tiles (list[ndarray]): The grid of images to concatenate.
        nx (int): The number of images per row.

    Returns:
        ndarray: The generated image.
    """
    return concatenate([concatenate(tiles[x:x+nx]) for x in range(0, len(tiles), nx)], axis=1)

In [ ]:
def shuffle_img(img: ndarray, min: int = 2) -> ndarray:
    """shuffle_img(img, min) splits img into a grid of images of the same size (at least min*min), then shuffles the grid and creates a new image.

    Args:
        img (ndarray): The image to shuffle.
        min (int, optional): The lowest bound of a sub-image width or height. Defaults to 2.

    Returns:
        ndarray: The generated image.
    """
    ly, lx, _ = img.shape
    dx, dy = max_div(lx, min), max_div(ly, min)
    tiles = list(parts(img, dx, dy))
    shuffle(tiles)
    return rebuild(tiles, lx // dx), (dx, dy)

In [ ]:
def get_min_size(images: Iterator[ndarray]) -> tuple[int, int]:
    """get_min_size(images) gets the smallest width and height of all images.

    Args:
        images (Iterator[ndarray]): The images to get the smallest size of.

    Returns:
        tuple[int, int]: The smallest width and height.
    """
    min_width, min_height = sys.maxsize, sys.maxsize
    for _, image in images:
        height, width, _ = image.shape
        min_width, min_height = min(min_width, width), min(min_height, height)
    return min_width, min_height

In [ ]:
def crop_center(image: ndarray, width: int, height: int) -> ndarray:
    """crop_center(image, w, h) removes pixels at both image borders to reduce it to size w*h.

    Args:
        image (ndarray): The image to crop.
        width (int): The target image width.
        height (int): The target image geight.

    Returns:
        ndarray: The cropped image.
    """
    old_height, old_width, _ = image.shape
    x0 = old_width // 2 - width // 2
    y0 = old_height // 2 - height // 2
    return image[y0:y0+height, x0:x0+width, :]

In [ ]:
def image_files(path: Path) -> Generator[tuple[Path, ndarray], None, None]:
    """image_files(path) yields all image file names and content in path.

    Args:
        path (Path): The path to get image files from.

    Yields:
        Generator[tuple[Path, ndarray], None, None]: The file names, and their associated content.
    """
    for file in path.glob('**/*'):
        if file.is_dir():
            continue
        yield file, imread(str(file))

In [ ]:
input_dir = './images'
output_dir = './masks'
hist_dir = './histograms'

input_path = Path(input_dir)
output_path = Path(output_dir)
hist_path = Path(hist_dir)
output_path.mkdir(parents=True, exist_ok=True)

reference = imread('ref.jpg')

min_tile_size = 30
min_image_size = get_min_size(image_files(input_path))

In [ ]:
for file, image in image_files(input_path):
    image = crop_center(image, *min_image_size)
    image = match_histograms(image, reference, channel_axis=-1)
    shuffled_image, res = shuffle_img(image, min_tile_size)

    imwrite(str(hist_path / file.relative_to(input_dir)), image)
    imwrite(str(output_path / file.relative_to(input_dir)), shuffled_image)

    print("Using tiles of size %s to shuffle '%s'" % (res, file))